## Домашнее задание

Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками с помощью LDA в gensim и NMF в sklearn**.


1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases,  можно также попробовать другие способы построить нграммы); 

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

6) между словарем и обучением модели добавьте tfidf (`gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`);

7) повторите пункт 4 на преобразованном корпусе;

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA.

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **

In [1]:
import gensim
import json
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

morph = MorphAnalyzer()

C:\Users\1\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Данные

In [36]:
#stops = set(stopwords.words('russian')) | {'gt',} - до изменения
stops = set(stopwords.words('russian')) | {'gt',} | set(stopwords.words('english'))
def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def normalize(words):
    norm_words = [morph.parse(word)[0].normal_form for word in words if len(set(word)) > 1]
    return norm_words

def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    words = [word.strip(string.punctuation) for word in text.split()]
    words = [word for word in words if word]
    
    return words

Возьмем 4 тыс статьи с Хабра. Это мало для хорошей тематической модели, но иначе у нас просто ничего не обучится за семинар.

In [ ]:
data = open('habr_texts.txt', 'r', encoding = 'UTF-8').read()

В текстах есть тэги. Потрем их. Ещё токенизируем самым простым способом и нормализуем Pymorphy.

In [ ]:
%%time
texts = open('habr_texts.txt').read().splitlines()
texts = [tokenize(remove_tags(text.lower())) for text in texts]

In [ ]:
%%time
texts = open('habr_texts.txt', encoding = 'UTF-8').read().splitlines()
texts = [normalize(tokenize(text.lower())) for text in texts]

In [3]:
%%time
texts = open('habr_texts.txt', encoding = 'UTF-8').read().splitlines()
texts = opt_normalize([tokenize(remove_tags(text.lower())) for text in texts], 30000)

Wall time: 11.6 s


### Тематическое моделирование в gensim

In [46]:
def gensim_lda (texts, no_above=0.3, num_topics=100, passes=10, eta = 'auto', iterations=20, tfidf=False, n_gram=True):

    if n_gram:
        ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4)
        p = gensim.models.phrases.Phraser(ph)
        texts = p[texts]

    # словарь для моделей
    dictionary = gensim.corpora.Dictionary(texts)
    dictionary.filter_extremes(no_above=no_above)
    dictionary.compactify()

    # тексты в мешки слов
    corpus = [dictionary.doc2bow(text) for text in texts]

    if tfidf:
        tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
        corpus = tfidf[corpus]

    model = gensim.models.LdaMulticore(corpus, num_topics=num_topics,
                                       id2word=dictionary, passes=passes, eta=eta, iterations=iterations, random_state = 137)
    return model, corpus

In [43]:
?gensim.models.LdaMulticore

In [26]:
%%time
lda, corpus = gensim_lda(texts) 

Wall time: 2min 27s


Посмотрим на топики.

In [27]:
lda.print_topics()

[(19,
  '0.035*"apple" + 0.013*"мобильный" + 0.008*"iphone" + 0.005*"устройство" + 0.005*"android" + 0.005*"ios" + 0.005*"технология" + 0.005*"рынок" + 0.004*"сервис" + 0.004*"смартфон"'),
 (11,
  '0.020*"файл" + 0.015*"сервер" + 0.008*"настройка" + 0.007*"скрипт" + 0.006*"сертификат" + 0.006*"установка" + 0.006*"добавить" + 0.005*"установить" + 0.005*"домен" + 0.005*"пароль"'),
 (88,
  '0.037*"автомобиль" + 0.012*"ms" + 0.011*"машина" + 0.010*"водитель" + 0.008*"дорога" + 0.007*"tesla" + 0.006*"скорость" + 0.005*"модель" + 0.005*"цена" + 0.004*"беспилотный_автомобиль"'),
 (41,
  '0.011*"сигнал" + 0.006*"распознавание" + 0.006*"дата-центр" + 0.005*"milliseconds" + 0.004*"тд" + 0.004*"цод" + 0.003*"объект" + 0.003*"метрика" + 0.003*"класс" + 0.003*"дц"'),
 (27,
  '0.012*"файл" + 0.006*"маршрутизатор" + 0.006*"electron" + 0.006*"программа" + 0.005*"lsa" + 0.005*"строка" + 0.005*"java" + 0.005*"метод" + 0.004*"номер" + 0.004*"скрипт"'),
 (55,
  '0.006*"учёный" + 0.006*"земля" + 0.004*"эне

In [18]:
import numpy as np

In [24]:
?lda.log_perplexity

In [28]:
lda.log_perplexity(corpus[:2000], total_docs=100)

-19.665877118389492

В целом, топики уже достаточно похожи на правду.
Если посмотреть на некоторые списки, то можно увидеть в топиках большое количество частей кода (что логично, это же Хабр). Это почти все слова на латинице, но полностью английский удалять нельзя, чтобы не исчезли важные термины, которым может быть посвящена статья. Поэтому просто очистим тексты от частотных английских слов и слов короче 5и букв.

In [37]:
texts = open('habr_texts.txt', encoding = 'UTF-8').read().splitlines()
texts = opt_normalize([tokenize(remove_tags(text.lower())) for text in texts], 30000)

In [38]:
for text in texts:
    for word in text:
        if len(word) <=4:
            text.remove(word)

In [39]:
%%time
lda, corpus = gensim_lda(texts)

Wall time: 2min


In [42]:
lda.print_topics()

[(88,
  '0.006*"временной" + 0.006*"попробовать" + 0.005*"находить" + 0.004*"задание" + 0.004*"директория" + 0.004*"параметр" + 0.004*"посмотреть" + 0.004*"2" + 0.004*"запустить" + 0.004*"модель"'),
 (44,
  '0.069*"модуль" + 0.012*"контроллер" + 0.007*"добавить" + 0.007*"страница" + 0.006*"ответ" + 0.006*"зависимость" + 0.005*"сообщение" + 0.005*"метод" + 0.005*"запрос" + 0.005*"загрузка"'),
 (8,
  '0.048*"0" + 0.033*"1" + 0.018*"else" + 0.017*"2" + 0.016*"0_0" + 0.014*"4" + 0.013*"return" + 0.013*"int" + 0.011*"5" + 0.010*"3"'),
 (35,
  '0.014*"безопасность" + 0.013*"атака" + 0.011*"доступ" + 0.010*"защита" + 0.008*"клиент" + 0.008*"интернет" + 0.008*"сервер" + 0.007*"оператор" + 0.007*"сервис" + 0.006*"злоумышленник"'),
 (77,
  '0.010*"метка" + 0.009*"маршрут" + 0.009*"пакет" + 0.009*"маршрутизатор" + 0.008*"значение" + 0.007*"трафик" + 0.006*"точка" + 0.006*"данный" + 0.005*"протокол" + 0.005*"интерфейс"'),
 (49,
  '0.018*"значение" + 0.012*"точка" + 0.011*"алгоритм" + 0.009*"элемен

In [41]:
lda.log_perplexity(corpus[:2000], total_docs=100)

-20.13378572284271

Построим нескролько LDA моделей.
Будем рандомно изменять три параметра: passes, eta и num_topics.

In [58]:
%%time
lda, corpus = gensim_lda(texts, eta = 0.01, num_topics = 50)
print(lda.log_perplexity(corpus[:2000], total_docs=100))

-30.559156468058315
Wall time: 1min 55s


In [60]:
lda.print_topics(10)

[(42,
  '0.013*"устройство" + 0.010*"intel" + 0.009*"linux" + 0.009*"компьютер" + 0.008*"программа" + 0.007*"память" + 0.006*"производительность" + 0.006*"процессор" + 0.006*"поддержка" + 0.005*"технология"'),
 (43,
  '0.017*"сервер" + 0.013*"настройка" + 0.009*"виртуальный_машина" + 0.008*"восстановление" + 0.007*"бэкап" + 0.006*"резервный_копирование" + 0.006*"отчёт" + 0.006*"номер" + 0.005*"доступ" + 0.005*"настроить"'),
 (28,
  '0.031*"точка" + 0.019*"объект" + 0.019*"карта" + 0.017*"координата" + 0.016*"текстура" + 0.012*"изображение" + 0.010*"алгоритм" + 0.007*"линия" + 0.006*"пиксель" + 0.006*"шейдёр"'),
 (32,
  '0.015*"студент" + 0.011*"makerbot" + 0.009*"университет_итмый" + 0.009*"экзамен" + 0.007*"знание" + 0.007*"город" + 0.006*"программирование" + 0.006*"материал" + 0.006*"университет" + 0.006*"олимпиада"'),
 (15,
  '0.020*"yahoo" + 0.015*"цифра" + 0.014*"биткоина" + 0.010*"openstack" + 0.010*"значение" + 0.008*"стрелка" + 0.007*"fedora" + 0.006*"fujitsu" + 0.006*"парсер" 

In [61]:
%%time
lda, corpus = gensim_lda(texts, eta = 0.5, num_topics = 200, passes = 5)
print(lda.log_perplexity(corpus[:2000], total_docs=100))

-18.982029105899358
Wall time: 2min 14s


In [62]:
lda.print_topics(10)

[(113,
  '0.005*"лампа" + 0.001*"вместо" + 0.001*"индекс_цветопередача" + 0.001*"elixir" + 0.001*"рубль" + 0.001*"продавать" + 0.001*"death" + 0.000*"остаться" + 0.000*"магазин" + 0.000*"лампочка"'),
 (136,
  '0.003*"станок" + 0.001*"jarvis" + 0.001*"гитхаб" + 0.001*"цукерберг" + 0.001*"коммит" + 0.001*"yum_install" + 0.001*"starttime" + 0.001*"объект" + 0.001*"schneider_electric" + 0.000*"библиотека"'),
 (134,
  '0.009*"сигнал" + 0.006*"учёный" + 0.006*"исследование" + 0.003*"метод" + 0.003*"вектор" + 0.003*"частота" + 0.003*"область" + 0.003*"пациент" + 0.003*"значение" + 0.003*"группа"'),
 (106,
  '0.007*"клиент" + 0.004*"продукт" + 0.004*"сотрудник" + 0.004*"сервис" + 0.004*"рынок" + 0.003*"что-то" + 0.003*"какой-то" + 0.003*"бизнес" + 0.003*"доклад" + 0.003*"заниматься"'),
 (70,
  '0.029*"робот" + 0.010*"земля" + 0.009*"астероид" + 0.008*"планета" + 0.005*"поверхность" + 0.004*"аппарат" + 0.004*"орбита" + 0.003*"планет" + 0.003*"учёный" + 0.003*"венера"'),
 (183,
  '0.002*"кальций

In [63]:
%%time
lda, corpus = gensim_lda(texts, eta = 1, num_topics = 50, passes = 5)
print(lda.log_perplexity(corpus[:2000], total_docs=100))

-17.707551236691305
Wall time: 1min 21s


In [64]:
lda.print_topics(10)

[(35,
  '0.005*"клиент" + 0.005*"сервис" + 0.005*"сервер" + 0.005*"доступ" + 0.004*"безопасность" + 0.004*"сотрудник" + 0.003*"ресурс" + 0.003*"атака" + 0.003*"организация" + 0.003*"защита"'),
 (7,
  '0.002*"нагрузка" + 0.002*"сервер" + 0.001*"устройство" + 0.001*"скорость" + 0.001*"redmine" + 0.001*"технология" + 0.001*"сервис" + 0.001*"запрос" + 0.001*"болезнь" + 0.001*"модель"'),
 (38,
  '0.006*"контейнер" + 0.004*"сервер" + 0.004*"docker" + 0.003*"server" + 0.003*"настройка" + 0.003*"конфигурация" + 0.002*"параметр" + 0.002*"директория" + 0.002*"изменение" + 0.002*"hadoop"'),
 (10,
  '0.003*"папка" + 0.002*"установка" + 0.002*"токен" + 0.002*"итератор" + 0.002*"сервер" + 0.001*"запустить" + 0.001*"список" + 0.001*"сборка" + 0.001*"ссылка" + 0.001*"установить"'),
 (46,
  '0.008*"книга" + 0.005*"программирование" + 0.003*"читать" + 0.003*"программа" + 0.003*"слово" + 0.003*"какой-то" + 0.002*"что-то" + 0.002*"pocketbook" + 0.002*"компьютер" + 0.002*"ребёнок"'),
 (3,
  '0.008*"игрок" 

In [65]:
%%time
lda, corpus = gensim_lda(texts, num_topics = 150, passes = 20)
print(lda.log_perplexity(corpus[:2000], total_docs=100))

-20.33758031022881
Wall time: 4min 47s


In [66]:
lda.print_topics(10)

[(55,
  '0.036*"доклад" + 0.035*"участник" + 0.028*"конференция" + 0.011*"рассказать" + 0.010*"блокчейн" + 0.010*"мероприятие" + 0.009*"выступление" + 0.008*"участие" + 0.008*"спикер" + 0.007*"встреча"'),
 (81,
  '0.025*"блокировка" + 0.007*"конкурс" + 0.007*"критический_секция" + 0.006*"лекция" + 0.005*"калькулятор" + 0.004*"трансляция" + 0.004*"материал" + 0.004*"ip" + 0.004*"байт" + 0.004*"круглый_скобка"'),
 (45,
  '0.024*"регистр" + 0.022*"angular" + 0.013*"трекинг" + 0.012*"значение" + 0.008*"webassembly" + 0.007*"инструкция" + 0.007*"изменение" + 0.005*"быстрый" + 0.005*"номер" + 0.005*"https"'),
 (22,
  '0.024*"email" + 0.014*"begin" + 0.010*"university" + 0.009*"algorithms" + 0.007*"recipients" + 0.007*"length" + 0.006*"introduction" + 0.006*"весна" + 0.006*"youtube" + 0.006*"quoted_identifier_create"'),
 (50,
  '0.063*"автомобиль" + 0.033*"машина" + 0.019*"водитель" + 0.010*"безопасность" + 0.009*"дорога" + 0.009*"управление" + 0.008*"технология" + 0.008*"устройство" + 0.007*

Лучше всего отработала модель [eta = 0.01, num_topics = 50].
Лучшие топики модели:
(13, '0.060*"камера" + 0.011*"видео" + 0.008*"сервер" + 0.007*"устройство" + 0.007*"сканирование" + 0.006*"покупка" + 0.006*"управление" + 0.005*"сервис" + 0.005*"google" + 0.005*"доступ"')
(42, '0.013*"устройство" + 0.010*"intel" + 0.009*"linux" + 0.009*"компьютер" + 0.008*"программа" + 0.007*"память" + 0.006*"производительность" + 0.006*"процессор" + 0.006*"поддержка" + 0.005*"технология"')
(28, '0.031*"точка" + 0.019*"объект" + 0.019*"карта" + 0.017*"координата" + 0.016*"текстура" + 0.012*"изображение" + 0.010*"алгоритм" + 0.007*"линия" + 0.006*"пиксель" + 0.006*"шейдёр"')

Добавим tfidf.

In [67]:
%%time
lda, corpus = gensim_lda(texts, eta = 0.01, num_topics = 50, tfidf=True)
print(lda.log_perplexity(corpus[:2000], total_docs=100))

-113.10003378914782
Wall time: 3min 45s


In [68]:
lda.print_topics(10)

[(0,
  '0.010*"видеокарта" + 0.008*"education" + 0.006*"юпитер" + 0.005*"ludum" + 0.005*"nvidia" + 0.005*"samsung_galaxy" + 0.004*"operations" + 0.004*"частота_ггц" + 0.003*"reads" + 0.003*"bluemix"'),
 (12,
  '0.005*"предупреждение_pvs-studio" + 0.005*"flussonic" + 0.004*"колонна" + 0.004*"baidu" + 0.004*"прямоугольник" + 0.003*"unreal_engine" + 0.003*"2d" + 0.003*"выпуск_подкаста" + 0.003*"weather" + 0.003*"всплывать"'),
 (17,
  '0.012*"ботнет" + 0.010*"infiniband" + 0.009*"mediatek" + 0.008*"mirai" + 0.007*"наушник" + 0.007*"fujitsu" + 0.006*"gt;&gt;&gt" + 0.006*"эксплойт" + 0.005*"обследование" + 0.005*"расширение_вселенная"'),
 (13,
  '0.017*"pvs-studio" + 0.008*"анализатор" + 0.005*"working" + 0.005*"param" + 0.004*"анализатор_pvs-studio" + 0.004*"config" + 0.004*"cmake" + 0.004*"await" + 0.004*"returns" + 0.004*"clock"'),
 (8,
  '0.007*"pebble" + 0.006*"protected_function" + 0.005*"литий" + 0.005*"фонарик" + 0.004*"makerbot" + 0.004*"fixedupdate" + 0.004*"dolby_atmos" + 0.004*"r

In [69]:
%%time
lda, corpus = gensim_lda(texts, eta = 0.5, num_topics = 200, passes = 5, tfidf=True)
print(lda.log_perplexity(corpus[:2000], total_docs=100))

-19.329379318414293
Wall time: 2min 55s


In [70]:
lda.print_topics(10)

[(113,
  '0.000*"стекло" + 0.000*"препарат" + 0.000*"пациент" + 0.000*"робот" + 0.000*"tesla" + 0.000*"математика" + 0.000*"сознание" + 0.000*"автомобиль" + 0.000*"death" + 0.000*"доказательство"'),
 (136,
  '0.000*"автомобиль" + 0.000*"водитель" + 0.000*"квеста" + 0.000*"javascript" + 0.000*"коммит" + 0.000*"декабрь" + 0.000*"проведение" + 0.000*"yield_return" + 0.000*"watson" + 0.000*"starcraft"'),
 (134,
  '0.000*"игрок" + 0.000*"nokia" + 0.000*"apple" + 0.000*"firebase" + 0.000*"сигнал" + 0.000*"патент" + 0.000*"гарнитур" + 0.000*"drone" + 0.000*"смартфон" + 0.000*"игровой"'),
 (106,
  '0.000*"javascript" + 0.000*"браузер" + 0.000*"нейрон" + 0.000*"react" + 0.000*"сервер" + 0.000*"устройство" + 0.000*"модель" + 0.000*"return" + 0.000*"google" + 0.000*"программа"'),
 (70,
  '0.000*"удовольствие_ответить" + 0.000*"web-интерфейс" + 0.000*"контрольный_точка" + 0.000*"asterisk" + 0.000*"позади" + 0.000*"следование" + 0.000*"земной" + 0.000*"сертификат" + 0.000*"продукт" + 0.000*"плеер"'

In [71]:
%%time
lda, corpus = gensim_lda(texts, eta = 1, num_topics = 50, passes = 5, tfidf=True)
print(lda.log_perplexity(corpus[:2000], total_docs=100))

-17.39506433241136
Wall time: 2min 14s


In [72]:
lda.print_topics(10)

[(35,
  '0.000*"устройство" + 0.000*"сборка" + 0.000*"пакет" + 0.000*"google" + 0.000*"операция" + 0.000*"номер" + 0.000*"калькулятор" + 0.000*"image" + 0.000*"электронный" + 0.000*"поддержка"'),
 (7,
  '0.000*"public" + 0.000*"num" + 0.000*"инструментарий" + 0.000*"медаль" + 0.000*"дата-центр" + 0.000*"semantic" + 0.000*"спутник" + 0.000*"react" + 0.000*"сервер" + 0.000*"bootstrap"'),
 (38,
  '0.000*"выгрузить" + 0.000*"резервный_копирование" + 0.000*"линейный" + 0.000*"0" + 0.000*"восстановление" + 0.000*"физика" + 0.000*"резервный_копия" + 0.000*"станция" + 0.000*"компонент" + 0.000*"виртуальный_машина"'),
 (10,
  '0.000*"устройство" + 0.000*"windows" + 0.000*"доклад" + 0.000*"усилитель" + 0.000*"сервер" + 0.000*"сообщение" + 0.000*"игрок" + 0.000*"книга" + 0.000*"linux" + 0.000*"видео"'),
 (46,
  '0.000*"спутник" + 0.000*"орбита" + 0.000*"земля" + 0.000*"станция" + 0.000*"планета" + 0.000*"ракета" + 0.000*"аппарат" + 0.000*"миссия" + 0.000*"астероид" + 0.000*"учёный"'),
 (3,
  '0.0

In [73]:
%%time
lda, corpus = gensim_lda(texts, num_topics = 150, passes = 20, tfidf=True)
print(lda.log_perplexity(corpus[:2000], total_docs=100))

-91.95206649599642
Wall time: 7min 21s


In [74]:
lda.print_topics(10)

[(55,
  '0.007*"resharper" + 0.006*"compute" + 0.006*"identifier" + 0.005*"backlog" + 0.004*"feature" + 0.004*"cordova" + 0.004*"navigation" + 0.003*"snapshot" + 0.003*"subscription" + 0.003*"неограниченный"'),
 (81,
  '0.008*"акустический_система" + 0.006*"лабиринт" + 0.006*"аудиосистема" + 0.006*"аудиомания" + 0.005*"наушник" + 0.005*"коэффициент_усиление" + 0.003*"выходной_мощность" + 0.003*"apk" + 0.003*"вебинар" + 0.002*"усилитель"'),
 (45,
  '0.014*"микросервис" + 0.007*"такси" + 0.005*"brocade" + 0.004*"systemjs" + 0.004*"notes" + 0.003*"background-size" + 0.003*"results" + 0.003*"emc" + 0.003*"phoenix" + 0.003*"цифровой_экономика"'),
 (22,
  '0.007*"лекция_лекция" + 0.005*"email_маркетинг" + 0.004*"кларк" + 0.004*"девяносто" + 0.003*"кодек" + 0.003*"контрольный_сумма" + 0.002*"р_р" + 0.002*"№" + 0.002*"чарльз" + 0.002*"гуглить"'),
 (50,
  '0.013*"беспилотный_автомобиль" + 0.009*"positive_technologies" + 0.005*"беспилотный" + 0.004*"mortal_kombat" + 0.004*"begin_begin" + 0.004*"

Результаты лучшей модели сильно ухудшились. Да и в остальных моделях результаты лучше не стали. Появилось много слов, которые явно не относятся к топикам. Некоторого мусора не было даже при самом первом запуске.

### Разложение матриц в sklearn

In [75]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

Sklearn принимает на вход строки, поэтому склеим наши списки.

In [76]:
stexts = [' '.join(text) for text in texts]

Сделаем матрицу слова-документы с помощью TfidfVectorizer

In [77]:
vectorizer = TfidfVectorizer(max_features=25000, min_df=5, max_df=0.3, lowercase=False)
X = vectorizer.fit_transform(stexts)

Разложим её.

In [78]:
model = NMF(n_components=30)

In [79]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=30, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [80]:
def get_nmf_topics(model, n_top_words):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(30):
        
        #топ n слов для темы.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [81]:
get_nmf_topics(model, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 21,Topic # 22,Topic # 23,Topic # 24,Topic # 25,Topic # 26,Topic # 27,Topic # 28,Topic # 29,Topic # 30
0,программист,return,обучение,сигнал,lt,устройство,public,атака,космический,игрок,...,google,объект,рынок,windows,клиент,дата,сервер,товар,страница,автомобиль
1,продукт,function,нейросеть,частота,gt,смартфон,string,безопасность,спутник,игра,...,android,класс,блокчейн,microsoft,услуга,центр,запрос,скидка,браузер,tesla
2,сотрудник,amp,машинный,усилитель,div,телефон,private,уязвимость,орбита,игровой,...,мобильный,значение,российский,linux,продукт,оборудование,сервис,магазин,элемент,беспилотный
3,программирование,const,изображение,канал,input,мобильный,class,злоумышленник,ракета,играть,...,реклама,метод,бизнес,studio,сервис,инфраструктура,пакет,покупатель,текст,машина
4,понимать,import,алгоритм,напряжение,body,аккумулятор,static,пароль,аппарат,персонаж,...,firebase,элемент,страна,visual,бизнес,услуга,скрипт,покупка,контент,водитель
5,деньга,false,нейронный,связь,std,гаджет,класс,защита,земля,геймплей,...,unity,переменный,технология,server,письмо,облачный,адрес,распродажа,вкладка,автопилот
6,думать,string,интеллект,диапазон,компонент,девайс,override,вредоносный,станция,комната,...,сервис,свойство,деньга,azure,звонок,сервер,настройка,продажа,реклама,дорога
7,жизнь,result,модель,антенна,li,умный,return,доступ,полёт,steam,...,apple,строка,россия,обновление,оператор,провайдер,домен,пятница,chrome,watson
8,понять,struct,искусственный,станция,class,беспроводный,метод,шифрование,космос,миссия,...,store,массив,рубль,программа,продажа,облако,сообщение,продавец,кнопка,автомобильный
9,знание,int,программа,искажение,элемент,датчик,import,информационный,планета,unity,...,swift,вызов,миллион,драйвер,интерфейс,охлаждение,сертификат,заказ,экран,грузовик


In [82]:
model = NMF(n_components=50, init='nndsvd', solver='mu')
model.fit(X)
get_nmf_topics(model, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 21,Topic # 22,Topic # 23,Topic # 24,Topic # 25,Topic # 26,Topic # 27,Topic # 28,Topic # 29,Topic # 30
0,сотрудник,return,учёный,устройство,lt,android,public,атака,космический,игрок,...,дата,объект,сервис,windows,обучение,товар,автомобиль,виртуальный,клиент,камера
1,бизнес,function,исследование,сигнал,gt,google,string,уязвимость,спутник,игра,...,центр,класс,сообщение,microsoft,нейросеть,скидка,беспилотный,машина,звонок,видео
2,программист,const,научный,датчик,div,смартфон,private,безопасность,ракета,игровой,...,оборудование,метод,облачный,linux,машинный,магазин,tesla,реальность,услуга,движение
3,деньга,false,температура,частота,input,мобильный,class,злоумышленник,орбита,играть,...,инфраструктура,значение,технология,server,нейронный,покупатель,машина,vmware,номер,регистратор
4,программирование,модуль,университет,плата,компонент,устройство,static,защита,аппарат,персонаж,...,услуга,свойство,telegram,azure,интеллект,распродажа,водитель,vsphere,оператор,ivideon
5,понимать,значение,исследователь,питание,std,apple,класс,вредоносный,земля,геймплей,...,облачный,элемент,платформа,браузер,алгоритм,покупка,автопилот,hyper,вызов,объектив
6,программа,строка,земля,напряжение,body,телефон,return,устройство,станция,комната,...,облако,массив,мессенджер,visual,модель,пятница,дорога,виртуализация,интерфейс,blackvue
7,готовый,вызов,животное,режим,li,экран,override,доступ,полёт,steam,...,охлаждение,переменный,facebook,обновление,искусственный,заказ,автомобильный,технология,клиентский,видеонаблюдение
8,инструмент,параметр,теория,корпус,class,iphone,метод,пароль,космос,миссия,...,провайдер,операция,канал,компьютер,google,продавец,грузовик,облако,сообщение,разрешение
9,понять,var,поверхность,канал,render,samsung,final,информационный,двигатель,unity,...,стойка,person,рынок,драйвер,изображение,продажа,motors,сетевой,техподдержка,съёмка


In [83]:
model = NMF(n_components=50, init='nndsvdar', solver='mu')
model.fit(X)
get_nmf_topics(model, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 21,Topic # 22,Topic # 23,Topic # 24,Topic # 25,Topic # 26,Topic # 27,Topic # 28,Topic # 29,Topic # 30
0,продукт,return,учёный,устройство,lt,android,public,атака,космический,игрок,...,google,объект,блокчейн,windows,обучение,товар,дата,виртуальный,клиент,камера
1,программист,function,исследование,сигнал,gt,мобильный,private,безопасность,спутник,игра,...,сервис,класс,технология,microsoft,нейросеть,скидка,центр,машина,услуга,видео
2,программирование,const,научный,датчик,div,устройство,string,защита,орбита,игровой,...,аккаунт,метод,транзакция,linux,машинный,магазин,оборудование,реальность,бизнес,движение
3,понимать,amp,исследователь,плата,input,дизайн,class,злоумышленник,ракета,играть,...,поисковый,значение,финансовый,server,алгоритм,покупатель,инфраструктура,технология,продажа,регистратор
4,думать,false,университет,частота,std,unity,static,вредоносный,аппарат,персонаж,...,корпорация,элемент,платформа,обновление,нейронный,покупка,облачный,vmware,заказчик,ivideon
5,понять,var,температура,питание,body,swift,return,устройство,земля,геймплей,...,amazon,свойство,биткойна,azure,модель,распродажа,услуга,облако,интерфейс,объектив
6,жизнь,int,животное,напряжение,компонент,apple,override,программа,станция,комната,...,chrome,вызов,криптовалюта,драйвер,интеллект,пятница,облако,виртуализация,клиентский,blackvue
7,деньга,struct,квантовый,режим,li,платформа,класс,пароль,полёт,steam,...,cloud,переменный,сервис,программа,искусственный,продавец,провайдер,vsphere,сервис,фотография
8,знание,error,теория,канал,элемент,дайджест,метод,информационный,космос,миссия,...,microsoft,массив,bitcoin,компьютер,изображение,продажа,охлаждение,hyper,звонок,видеонаблюдение
9,вообще,else,эксперимент,температура,class,библиотека,final,ботнет,двигатель,история,...,реклама,операция,blockchain,платформа,распознавание,заказ,сервис,облачный,продукт,разрешение


In [84]:
model = NMF(n_components=100, init='nndsvd')
model.fit(X)
get_nmf_topics(model, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 21,Topic # 22,Topic # 23,Topic # 24,Topic # 25,Topic # 26,Topic # 27,Topic # 28,Topic # 29,Topic # 30
0,модель,lt,галактика,устройство,gt,кнопка,public,учёный,аппарат,игрок,...,дата,класс,порекомендовать,windows,обучение,товар,сотрудник,виртуальный,робот,камера
1,моделирование,div,звезда,умный,body,экран,private,исследование,земля,игра,...,центр,метод,предпочитать,microsoft,нейросеть,скидка,рабочий,машина,робототехника,видео
2,контекст,std,телескоп,девайс,bitcoin,интерфейс,string,исследователь,космический,игровой,...,оборудование,вызов,профессиональный,linux,машинный,магазин,бизнес,реальность,машина,ivideon
3,архитектура,return,солнце,гаджет,summary,панель,static,научный,орбита,играть,...,инфраструктура,string,слушать,server,интеллект,покупатель,руководитель,vmware,датчик,объектив
4,pocketbook,input,вселенная,мобильный,echo,вкладка,class,университет,планета,персонаж,...,охлаждение,class,пользоваться,azure,нейронный,распродажа,работник,vsphere,способный,видеонаблюдение
5,сознание,int,светов,беспроводный,config,нажатие,override,животное,спутник,геймплей,...,стойка,свойство,менеджер,обновление,искусственный,покупка,менеджер,hyper,рекорд,движение
6,представление,gt,миллиард,управление,script,настройка,final,группа,астероид,миссия,...,мониторинг,реализация,музыка,драйвер,deepmind,пятница,корпоративный,виртуализация,сборка,разрешение
7,программный,li,млечный,датчик,head,нажать,return,эксперимент,миссия,комната,...,серверный,экземпляр,бумажный,операционный,агент,заказ,директор,технология,скорость,съёмка
8,предметный,param,планета,производитель,html,клавиатура,new,обнаружить,поверхность,steam,...,воздух,value,заниматься,платформа,алгоритм,продавец,зарплата,oculus,дронов,аналоговый
9,область,class,волна,передача,latitude,иконка,using,социальный,солнечный,противник,...,эксплуатация,object,читалка,поддержка,нейрон,продажа,отпуск,производительность,result,gopro


In [85]:
model = NMF(n_components=100, init='nndsvdar')
model.fit(X)
get_nmf_topics(model, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 21,Topic # 22,Topic # 23,Topic # 24,Topic # 25,Topic # 26,Topic # 27,Topic # 28,Topic # 29,Topic # 30
0,программист,lt,аппарат,устройство,react,смартфон,public,нейросеть,станция,игрок,...,дата,объект,порекомендовать,windows,обучение,товар,сотрудник,виртуальный,клиент,камера
1,программирование,div,земля,умный,компонент,xiaomi,private,распознавание,базовый,игра,...,центр,модель,предпочитать,microsoft,машинный,скидка,рабочий,машина,услуга,видео
2,знание,std,космический,девайс,props,аккумулятор,string,обучение,связь,игровой,...,оборудование,свойство,профессиональный,linux,модель,магазин,бизнес,реальность,бизнес,ivideon
3,навык,return,орбита,беспроводный,render,дисплей,static,интеллект,антенна,играть,...,инфраструктура,person,слушать,server,нейронный,покупатель,руководитель,vmware,клиентский,объектив
4,математика,input,планета,гаджет,div,экран,class,deepmind,линия,миссия,...,охлаждение,менять,пользоваться,azure,алгоритм,покупка,менеджер,vsphere,продажа,видеонаблюдение
5,python,int,спутник,управление,компонента,гб,override,искусственный,иначе,комната,...,стойка,моделирование,менеджер,обновление,интеллект,распродажа,работник,hyper,заказчик,движение
6,язык,gt,астероид,мобильный,component,телефон,final,научный,космический,персонаж,...,мониторинг,астероид,музыка,драйвер,нейрон,пятница,корпоративный,виртуализация,техподдержка,разрешение
7,специалист,li,миссия,производитель,свойство,lenovo,new,агент,рабочий,геймплей,...,услуга,операция,бумажный,операционный,искусственный,заказ,директор,производительность,поддержка,съёмка
8,технический,param,поверхность,передача,button,планшет,return,исследователь,космос,прохождение,...,серверный,местоположение,заниматься,поддержка,признак,продавец,зарплата,virtual,crm,аналоговый
9,область,class,солнечный,mediatek,состояние,samsung,using,распознавать,оборудование,предмет,...,воздух,событие,читалка,платформа,агент,продажа,отпуск,сетевой,интерфейс,gopro


Лучше всего справилась модель [n_components=100, init='nndsvdar']. Она даже смогла выташить что-то более-менее адекватное из 5 го топика.
Лучшие топики: 24, 26, 29.